# Lecture : Text pre-processing, non-negative matrix factorization and topic modeling.

In this lecture we'll discuss
- Text-preprocessing techniques such as stemming and Tf-Idf.
- Non-negative matrix factorization.
- Some techniques for searching a document data set for matches to a query document.
- Extracting meaningful topics from a corpus of documents.

Note that **semantic** means "relating to meaning in language or logic." 

In [1]:
import numpy as np
import sklearn
from sklearn.datasets import fetch_20newsgroups 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.decomposition import NMF
import nltk
import re
import scipy.optimize.nnls

In [4]:
# Load the data
from nltk.tokenize import word_tokenize
data= fetch_20newsgroups(remove=('headers', 'footers', 'quotes')).data
print(data[2])

well folks, my mac plus finally gave up the ghost this weekend after
starting life as a 512k way back in 1985.  sooo, i'm in the market for a
new machine a bit sooner than i intended to be...

i'm looking into picking up a powerbook 160 or maybe 180 and have a bunch
of questions that (hopefully) somebody can answer:

* does anybody know any dirt on when the next round of powerbook
introductions are expected?  i'd heard the 185c was supposed to make an
appearence "this summer" but haven't heard anymore on it - and since i
don't have access to macleak, i was wondering if anybody out there had
more info...

* has anybody heard rumors about price drops to the powerbook line like the
ones the duo's just went through recently?

* what's the impression of the display on the 180?  i could probably swing
a 180 if i got the 80Mb disk rather than the 120, but i don't really have
a feel for how much "better" the display is (yea, it looks great in the
store, but is that all "wow" or is it really th

We'll use the bag-of-words approach. This means that we turn each article into an unordered list of words.

In [9]:
# Turn this article into a list of words
Test_Article = word_tokenize(data[0])
print(Test_Article)

['I', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'I', 'saw', 'the', 'other', 'day', '.', 'It', 'was', 'a', '2-door', 'sports', 'car', ',', 'looked', 'to', 'be', 'from', 'the', 'late', '60s/', 'early', '70s', '.', 'It', 'was', 'called', 'a', 'Bricklin', '.', 'The', 'doors', 'were', 'really', 'small', '.', 'In', 'addition', ',', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', '.', 'This', 'is', 'all', 'I', 'know', '.', 'If', 'anyone', 'can', 'tellme', 'a', 'model', 'name', ',', 'engine', 'specs', ',', 'years', 'of', 'production', ',', 'where', 'this', 'car', 'is', 'made', ',', 'history', ',', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', ',', 'please', 'e-mail', '.']


## Stop words

- These are commonly used words that have little semantic value.
- We typically remove them before performing any text analysis.
- The definition of a stop word is not entirely rigourous, but different languages have different freely downloadable lists of stop words


In [10]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
#print(stop_words)
Test_Article = [w.lower() for w in Test_Article]
#print(Test_Article)
Trimmed_Article = [w for w in Test_Article if not w in stop_words] 
print(Trimmed_Article)

['wondering', 'anyone', 'could', 'enlighten', 'car', 'saw', 'day', '.', '2-door', 'sports', 'car', ',', 'looked', 'late', '60s/', 'early', '70s', '.', 'called', 'bricklin', '.', 'doors', 'really', 'small', '.', 'addition', ',', 'front', 'bumper', 'separate', 'rest', 'body', '.', 'know', '.', 'anyone', 'tellme', 'model', 'name', ',', 'engine', 'specs', ',', 'years', 'production', ',', 'car', 'made', ',', 'history', ',', 'whatever', 'info', 'funky', 'looking', 'car', ',', 'please', 'e-mail', '.']


In [11]:
Trimmed_Article = [w for w in Trimmed_Article if not w=='.']
Trimmed_Article = [w for w in Trimmed_Article if not w== ',']
Trimmed_Article = [w for w in Trimmed_Article if not w== "n't"]
print(Trimmed_Article)

['wondering', 'anyone', 'could', 'enlighten', 'car', 'saw', 'day', '2-door', 'sports', 'car', 'looked', 'late', '60s/', 'early', '70s', 'called', 'bricklin', 'doors', 'really', 'small', 'addition', 'front', 'bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'production', 'car', 'made', 'history', 'whatever', 'info', 'funky', 'looking', 'car', 'please', 'e-mail']


## Stemming
- In English, the same base word can serve different grammatical roles.
- E.g. "swim" can be made into "swimming", "swimmers" and "swims"
- All of these words are the same semantically, that is they relate to the same topic or have the same meaning.
- Stemming is the process of removing the suffixes, or ends of words, in a principled manner.
- E.g. "swimming", "swimmers" and "swims" should all be stemmed to "swims"


In [17]:
#Load the stemmer function
porter = nltk.PorterStemmer()
porter.stem('swimming')

'swim'

In [18]:
# Now lets stem the article
Trimmed_Article  = [porter.stem(w) for w in Trimmed_Article]
print(Trimmed_Article)

['wonder', 'anyon', 'could', 'enlighten', 'car', 'saw', 'day', '2-door', 'sport', 'car', 'look', 'late', '60s/', 'earli', '70', 'call', 'bricklin', 'door', 'realli', 'small', 'addit', 'front', 'bumper', 'separ', 'rest', 'bodi', 'know', 'anyon', 'tellm', 'model', 'name', 'engin', 'spec', 'year', 'product', 'car', 'made', 'histori', 'whatev', 'info', 'funki', 'look', 'car', 'pleas', 'e-mail']


## Discussion
 - Numbers?
 - some stop words may have snuck through

## Vectorizing the corpus
- **Terminology:** a collection of documents is called a *corpus*.
- After stemming and removing stop words from our articles, we would like to **vectorize**
- **vectorize:** Turn document $d_{j}$ into vector $\mathbf{a}_j$.
- **Goal** convert a corpus to a matrix $A$ where columns correspond to articles and rows to key-words.
- **Question** How to choose keywords?
- **Question** How to fill in $A_{ij}$?

![alt text](term_document.jpg)



### Choosing key-words
- As we've already removed stop-words, the remaining words are likely to be informative.
- Compute the *corpus frequency* of all remaining words.
- Choose the $d$ most frequently occuring words as keywords.

![alt text](Word_Frequency.png)

### Term weighting

- Call the keywords $w_1,\ldots, w_{d}$. 
- Count up the number of times $w_i$ appears in $d_j$.
- If $w_i$ appears in $d_j$ frequently, it likely is telling us something about $d_j$.
- On the other hand, if $w_i$ appears in all $d_j$, it is likely telling us something about the *corpus*, and therefore is not useful in distinguishing articles.
- **Solution:** tf-idf weighting
    - $\text{tf}(w_i,d_j) = \# \text{times $w_i$ appears in $d_j$}$ 
    - $\text{df}(w_i) = \# \text{articles containing $w_i$ at least once}$.
    - $n = \# \text{of documents}$ 
    - $\displaystyle A_{ij} = \text{tf}(w_i,d_j)\log\left(\frac{n}{\text{df}(w_i)}\right)$
- Typically **normalize** columns: Replace $\mathbf{a}_j$ with $\mathbf{a}_j/\|\mathbf{a}_j\|_2$

>**Important:** Note that $A\in\mathbb{R}^{d\times n}$ is *non-negative* 

>**Important:** Note that *columns* of $A$ correspond to data points. This is the usual convention in topic modeling, but different to the convention for PCA.
    

## In Practice
This is all handled by the following two lines of code:

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer 
vectorizer = TfidfVectorizer(max_features=2000, min_df=10, stop_words='english')
# max features = max_number of keywords to consider.
# min_df = potential keywords with document frequency less than 10 will be ignored. This means we may get fewer than
# max_features keywords
A = vectorizer.fit_transform(data)
# Above line actually does the tf-idf conversion
idx_to_word = np.array(vectorizer.get_feature_names())
# Above line will get and store the keywords w_i
print(A.shape)

(11314, 2000)


>**Important:** Note that by convention scikit learn puts data points as *rows* of $A$ 

In [20]:
for word in idx_to_word:
    print(word)

00
000
01
02
03
04
05
06
0d
0t
10
100
1000
11
12
128
13
130
14
145
15
150
16
17
18
19
1988
1989
1990
1991
1992
1993
1d9
1st
1t
20
200
21
22
23
24
25
250
256
26
27
28
29
2nd
2tm
30
300
31
32
33
34
34u
35
36
37
38
386
39
3d
3l
3rd
3t
40
400
41
42
43
44
45
46
47
48
486
49
4t
50
500
51
52
53
54
55
56
57
58
59
5u
60
600
61
62
63
64
65
66
67
68
6ei
6g
70
71
72
73
74
75
75u
76
77
78
79
7ey
7u
80
800
81
82
84
85
86
88
89
90
91
92
93
95
99
9f
9v
__
a86
ability
able
absolute
absolutely
abuse
ac
accept
accepted
access
according
account
accurate
act
action
actions
active
activities
activity
acts
actual
actually
ad
add
added
addition
additional
address
addresses
administration
admit
advance
advantage
advice
afraid
age
agencies
agency
agents
ago
agree
ah
ahead
aid
aids
air
al
algorithm
alive
allow
allowed
allows
alt
alternative
amendment
america
american
americans
amiga
analysis
andrew
angeles
animals
announced
annual
anonymous
answer
answers
anti
anybody
apartment
app
apparently
appear
appears
appl

vs
wait
waiting
wall
want
wanted
wants
war
warning
washington
wasn
waste
watch
water
way
ways
weak
weapon
weapons
week
weeks
weight
welcome
went
west
western
white
wide
widget
widgets
wife
willing
win
window
windows
wings
winning
wire
wiring
wish
wm
woman
women
won
wonder
wondering
word
words
work
worked
working
works
world
worry
worse
worth
wouldn
write
writes
writing
written
wrong
wrote
wt
ww
x11
x11r5
xlib
xt
xterm
yeah
year
years
yes
yesterday
york
young
zip
zone


## Topic Modeling
- Even if there are millions of documents $\mathbf{a}_i\in\mathbb{R}^{d}$ there is likely only a small number of topics $\mathbf{w}_1,\ldots,\mathbf{w}_r\in\mathbb{R}^{d}$.
- **Hypothesis** each document is (approximately) a linear combination of topics: $\mathbf{a}_i = \sum_{\ell=1}^{r}\alpha_{\ell}\mathbf{w}_{\ell}$.
- Important that $\mathbf{w}_{\ell}\in\mathbb{R}^{d}$ are non-negative. Why?

![alt text](Topic_Modelling.png) 
 - Image from [here](https://towardsdatascience.com/topic-modeling-quora-questions-with-lda-nmf-aff8dce5e1dd)


## Non-negative Matrix Factorization

- Suppose that $A\in\mathbb{R}^{d\times n}$. 
- Let $\mathbb{R}^{d\times r}_{+}$ denote all $d\times r$ matrices with non-negative entries.
- Similarly for $\mathbb{R}^{r\times n}_{+}$
- Want $W\in \mathbb{R}^{d\times r}_{+}$ and $H \in \mathbb{R}^{r\times n}_{+}$ such that $\|A - WH\|_{F}$ is as small as possible.
- **NMF:** $ W,H = \text{argmin}_{U\in \mathbb{R}^{d\times r}_{+}, V\in\mathbb{R}^{r\times n}_{+}}\|A - UV\|^{2}_{F}$

*****
### Properties of NMF

 - Cannot expect $A = WH$ (like for SVD). Usually write $A\approx WH$.
 - NMF problem does not have a unique solution (like best rank $k$ approx. problem does). 
*****

### Algorithms for NMF
 - Suppose we know $W$. Then finding $H$ becomes (non-negative) matrix least squares:
 
 $ H = \text{argmin}_{V\in\mathbb{R}^{r\times n}_{+}}\|A - WV\|^{2}_{F}$
 
 - What if we knew $H$? Then finding $W$ is also (non-negative) matrix least squares:
 
 $W = \text{argmin}_{U\in\mathbb{R}^{d\times r}_{+}}\|A - UH\|^{2}_{F}$ 
 
 - **Idea:** If we have approximations $W_{k},H_{k}$:
     + Use $W_{k}$ to find new approx. of $H$:
         $ H_{k+1} = \text{argmin}_{V\in\mathbb{R}^{r\times n}_{+}}\|A - W_{k}V\|^{2}_{F} = \text{argmin}_{\mathbf{v}_{i} \geq 0 \ \text{ for } j=1,\ldots n} \sum_{j=1}^{n}\|\mathbf{a}_{j} - W_{k}\mathbf{v}_j\|_{2}^{2}$
     + Note that $V = \left[\begin{matrix} \mathbf{v}_1 & \ldots & \mathbf{v}_{r} \end{matrix}\right]$
     + $\mathbf{v}_{i} = \text{nnls}(W_{k},\mathbf{a}_i)$ for $i=1,\ldots, n$
     + $\|A - W_{k}V\|^{2}_{F} = \sum_{i=1}^{n}\sum_{j=1}^{d}\left(A_{ij} - \left(W_{k}V\right)_{ij}\right)^{2}$
     + Can do some rewriting: $A_{ij} = \mathbf{a}_{i,j}$
     + Then, use $H_{k+1}$ to find a new approx. of $W$:
         $ W_{k+1} = \text{argmin}_{U\in\mathbb{R}^{d\times r}_{+}}\|A - UH_{k+1}\|^{2}_{F}$
     + Extra steps: Know that $\|A - UH_{k+1}\|^{2}_{F} = \|\left(A - UH_{k+1}\right)^{\top}\|^{2}_{F} = \|A^{\top} - H_{k+1}^{\top}U^{\top}\|^{2}_{F}$
     + $\mathbf{u}_i = \text{nnls}(H_{k+1}^{\top},\mathbf{r}_i)$ where $\mathbf{r}_i$ is the $i$-th row of $A$
     + Repeat.

- This is **Alternating (Non-negative) Least Squares** (ALS).
- *Will implement this algorithm when we discuss projected gradient descent*
- See also *Algorithms for non-negative matrix factorization* by Lee and Heung (NIPS 2001) for a faster algorithm. 

**********

### NMF for Topic Modeling

- Suppose that $A\in\mathbb{R}^{d\times n}$ with *columns* corresponding to documents/ data points.
- Assume we have found $A \approx WH$ with $W\in \mathbb{R}^{d\times r}_{+}$ and $H \in \mathbb{R}^{r\times n}_{+}$.
- *Columns* of $W$ represent topics.
- For each column $\mathbf{a}_j$ can write $\mathbf{a}_{j} \approx \sum_{i=1}^{r} H_{ij}\mathbf{w}_i$. 
- $H_{ij} = $ correlation of document $j$ with topic $i$.
- Choosing $r$ is a bit of an art.




In [21]:
# apply NMF
nmf = NMF(n_components=20, solver="mu")
H = nmf.fit_transform(A)
W = nmf.components_
 
# print the topics
 
for i, topic in enumerate(W):
 
    print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in idx_to_word[topic.argsort()[-10:]]])))

Topic 1: really,say,way,said,right,did,ve,good,time,people
Topic 2: appreciated,hi,information,program,need,software,looking,help,use,mail
Topic 3: lord,church,christians,christian,believe,faith,christ,bible,jesus,god
Topic 4: algorithm,public,use,escrow,government,keys,clipper,encryption,chip,key
Topic 5: problem,cd,floppy,controller,ide,hard,disk,drives,scsi,drive
Topic 6: 15,20,price,condition,offer,shipping,10,new,sale,00
Topic 7: directory,version,problem,ms,program,running,files,dos,file,windows
Topic 8: player,win,hockey,play,season,players,year,games,team,game
Topic 9: colorado,pub,cc,university,cs,soon,banks,gordon,pitt,edu
Topic 10: new,oil,speed,dealer,miles,good,engine,bike,cars,car
Topic 11: ram,color,driver,vga,bus,cards,drivers,monitor,video,card
Topic 12: exactly,exist,say,work,new,doesn,mean,anybody,know,does
Topic 13: sorry,anybody,good,need,let,want,think,people,know,don
Topic 14: appreciate,wondering,got,info,tell,ve,hi,mail,advance,thanks
Topic 15: new,wanted,mean,

## Document Retrieval
- **Problem:**
    - Given corpus $\mathcal{D} = \{d_1,\ldots, d_n\}$.
    - Suppose a *new document* $q_1$ arrives.  
    - **Goal** return $a_{i}$ which is the best semantic match for $q_1$

- Typically, vectorize $\mathcal{D}$ to $A$ **offline**.
- **In practice** want to return match quickly ($\sim 0.1 $ seconds)

### Approach 1
 - Recall documents have been **vectorized** to $\mathbf{a}_1,\ldots,\mathbf{a}_n\in\mathbb{R}^{d}$ 
 - Vectorize $q_1$ in the same manner: $q_1 \to \mathbf{y}_1\in\mathbb{R}^{d}$
 - Return article which is **closest** to $\mathbf{y}_1$.
 - We can be flexible with how we measure closest:
     - $\ell_2$ or Euclidean distance: $\|\mathbf{y}_1 - \mathbf{a}_i\|$.
     - Cosine distance: $\frac{\mathbf{y}^{\top}\mathbf{a}_i}{\|\mathbf{y}\|_2\|\mathbf{a}_i\|_2}$ 
 - **Question:** When might cosine distance be preferable? 
 
 *****
 
 ### Approach 2
- Use the **Semantic Structure**
- Suppose we have nmf: $A \approx WH$ for NMF) 
- As established earlier, this means that $\mathbf{a}_{j} \approx \sum_{i=1}^{r}H_{ij}\mathbf{w}_i$
- Let $\mathbf{h}_{j}$ denote $j$-th *column* of $H$. Then $\sum_{i=1}^{r}H_{ij}\mathbf{w}_i = h_{j,1}\mathbf{w}_{1} + h_{j,2}\mathbf{w}_{2} + \ldots + h_{j,r}\mathbf{w}_{r}$. 
- **Low dimensional embedding:** $\mathbf{a}_{j} \mapsto \mathbf{h}_{j}\in\mathbb{R}^{r}$
- Can be more informative to measure distance in *topic space* $\mathbb{R}^{r}$. 
- **Idea** Project $\mathbf{y}_1$ to the *topic space* first, then measure distance.

****

Recommended further reading: [Word Mover's Distance](http://proceedings.mlr.press/v37/kusnerb15.pdf?source=post_page---------------------------)

In [3]:
import numpy as np
v1 = np.random.randn(4)
v2 = np.random.randn(4)
v3 = np.random.randn(4)
v4 = np.random.randn(4)

NameError: name 'scipy' is not defined